# Doğal Dil İşleme + Karar Bilimi (Natural Language Processing + Decision Science)

👩🏻‍🏫 Bu görevde şunları bir araya getireceğiz:
* 🗣 Doğal Dil İşleme (Natural Language Processing)
* 📊 Karar Bilimi (Decision Science)

🎯 Amaç, Olist üzerindeki ürünlerin ve satıcıların **olumsuz (kötü) yorumlarını** anlamaktır.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Data Manipulation
import numpy as np
import pandas as pd
pd.set_option("display.max_columns",None)

# Machine Learning
from sklearn.pipeline import make_pipeline

# Language Processing
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import string
import unidecode as unidecode

# Vectorizers and NLP Models
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [3]:
# Gerekli NLTK paketleri (Eksik varsa indirelim)
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp') # Portekizce Stemmer için gerekli!

[nltk_data] Downloading package stopwords to /home/tumay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/tumay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to /home/tumay/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

🕵🏻‍♂️ Olist’in CEO’su [Tiago Dalvi](https://www.linkedin.com/in/tiagodalvi/)’nin senden yorumları okuyup anlamanı istediğini hayal et.

- Müşteriler siparişlerini **1**, **2** veya **3** puanla değerlendirdiklerinde ne söylediler?
- En sık karşılaşılan olumsuz yorumlar neler?
    - En kötü puanlanan ürünler hakkında?
    - En kötü puanlanan satıcılar hakkında?


## (0) Kurulum 🔨

Öncelikle, Olist incelemeleriyle ilgili tüm bilgileri içeren DataFrame'i yükleyeceğiz!

In [4]:
df = pd.read_csv("https://d32aokrjazspmn.cloudfront.net/materials/olist_reviews.csv")

In [5]:
df.head()

,review_id,length_review,review_score,order_id,product_category_name,full_review
0,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,Recebi bem antes do prazo estipulado.
1,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,Parabéns lojas lannister adorei comprar pela ...
2,8670d52e15e00043ae7de4c01cc2fe06,174,4,b9bf720beb4ab3728760088589c62129,eletroportateis,recomendo aparelho eficiente. no site a marca ...
3,4b49719c8a200003f700d3d986ea1a19,45,4,9d6f15f95d01e79bd1349cc208361f09,beleza_saude,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
4,3948b09f7c818e2d86c9a546758b2335,56,5,e51478e7e277a83743b6f9991dbfa3fb,informatica_acessorios,"Super recomendo Vendedor confiável, produto ok..."


## (2) Metin Temizleme (Text Cleaning)

🧹 `cleaning(sentence)` işlevini oluşturun ve yorumlara uygulayın. **NLTK'da Portekizce lemmatizer bulunmadığını unutmayın** (genellikle bulunmaz, ancak `nltk.stem.RSLPStemmer` kök bulucu vardır).

In [7]:
from nltk.stem import RSLPStemmer

def cleaning(sentence):
    # 1. Veri tipi kontrolü (Eğer yorum boşsa veya sayıysa hata vermesin diye)
    if not isinstance(sentence, str):
        return ""
    
    # 2. Temel temizlik (Boşluk silme, Küçük harf)
    sentence = sentence.strip().lower()
    
    # 3. Aksanları kaldırma (unidecode kütüphanesi ile)
    # Örn: 'coração' -> 'coracao' olur.
    sentence = unidecode.unidecode(sentence)
    
    # 4. Sayıları kaldırma
    sentence = ''.join(char for char in sentence if not char.isdigit())
    
    # 5. Noktalama işaretlerini kaldırma
    sentence = ''.join(char for char in sentence if char not in string.punctuation)
    
    # 6. Tokenize (Kelimelere ayırma)
    tokens = word_tokenize(sentence)
    
    # 7. Stopwords ve Stemming
    stemmer = RSLPStemmer()
    stop_words = set(stopwords.words('portuguese')) # Dikkat: 'portuguese' seçtik
    
    cleaned_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    
    return " ".join(cleaned_tokens)

# Fonksiyonu uygulama
# Bu işlem biraz uzun sürebilir (Portekizce Stemmer biraz ağırdır)
df['clean_text'] = df['full_review'].apply(cleaning)

# Sonucu kontrol et
df[['full_review', 'clean_text']].head()

,full_review,clean_text
0,Recebi bem antes do prazo estipulado.,receb bem ant praz estipul
1,Parabéns lojas lannister adorei comprar pela ...,parab loj lannist ador compr internet segur pr...
2,recomendo aparelho eficiente. no site a marca ...,recom aparelh efici sit marc aparelh impress d...
3,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",pouc travandopel val ta boa
4,"Super recomendo Vendedor confiável, produto ok...",sup recom vend confia produt ok entreg ant praz


In [ ]:
# SENİN KODUN BURAYA

## (3) Kötü yorumların analizi

### (3.1) Düşük inceleme puanlarına sahip veri kümesi

😱 1 ile 3 arasında puan alan yorumların oranı nedir? 

In [8]:
# 1, 2 veya 3 puan verenlerin sayısı
bad_reviews_count = len(df[df['review_score'] <= 3])

# Toplam yorum sayısı
total_reviews_count = len(df)

# Oranı hesapla
ratio = bad_reviews_count / total_reviews_count

print(f"Kötü yorumların oranı: %{ratio*100:.2f}")

Kötü yorumların oranı: %26.72


🕵🏻‍♂️ Bu yorumlara odaklanalım...

In [9]:
# Sadece puanı 3'ten küçük veya eşit olanları al
bad_reviews = df[df['review_score'] <= 3].copy()

# Temizlenmiş metni boş (NaN veya boş string) olan satırları atalım
# Çünkü yorum yazmamışsa analiz yapamayız.
bad_reviews = bad_reviews[bad_reviews['clean_text'].notnull()]
bad_reviews = bad_reviews[bad_reviews['clean_text'] != ""]

# Kontrol edelim
print(f"İncelenecek kötü yorum sayısı: {bad_reviews.shape[0]}")
bad_reviews.head()

İncelenecek kötü yorum sayısı: 9632


,review_id,length_review,review_score,order_id,product_category_name,full_review,clean_text
14,e233e51d11511bf30e568c76360ace52,97,1,548df2c6e5f089574614894bca78acf5,eletronicos,recebi somente 1 controle Midea Split ESTILO....,receb soment control mide split estil falt con...
23,8b230a1373c6dc4bd867099fda1d7039,60,3,071251fe3b3493294536f03737a8a679,ferramentas_jardim,Eu comprei duas unidades e só recebi uma e ag...,compr dua unidad so receb agor fac
24,cb2fc3e5711b5ae85e0491ee18af63ed,72,3,34e6d418f368f8079ae152bc178bc66a,beleza_saude,"Produto bom, porém o que veio para mim não co...",produt bom por vei mim nao condiz fot anunci
25,60c714ed14cef913944a3147094a4742,36,1,9ac05114800f02bfaa783bd76842dbe2,moveis_decoracao,"Produto muito inferior, mal acabado.",produt inferi mal acab
28,0bd4dcc4f6c4621baf37f73495cad8c4,16,3,a11cd01ac67beef7e8bf09740d8a35c1,esporte_lazer,Entrega no prazo,entreg praz


### (3.2) Vektörleştirme

🔡 ➡️ 🔢 Metinlerini vektörleştir.

- **Bigram**’leri (iki kelimelik ifadeler) mutlaka hesaba kat.
- Çok sık geçen kelimeleri çıkarmak için `max_df = 0.75` ayarla.
- Spoiler uyarısı: Sonunda **20.000+** kelimeye ulaşacaksın…  
  Bu challenge için sadece `max_features = 5000` ile sınırla.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

# 1. Vektörleştiriciyi Ayarla
# ngram_range=(2, 2) -> Sadece ikili kelime gruplarını al (Örn: "teslim edilmedi")
# max_df=0.75 -> %75'ten fazla geçen çok yaygın kelimeleri at
# max_features=5000 -> Sadece en popüler 5000 ifadeyi tut
vectorizer = CountVectorizer(ngram_range=(2, 2), max_df=0.75, max_features=5000)

# 2. Metni Sayıya Çevir (Fit & Transform)
# bad_reviews tablosundaki 'clean_text' sütununu kullanıyoruz
data_vectorized = vectorizer.fit_transform(bad_reviews['clean_text'])

# 3. Kontrol (Matris boyutuna bakalım)
print("Vektörleştirilmiş veri boyutu:", data_vectorized.shape)
# Beklenen: (9632, 5000) gibi bir şey olmalı (Satır sayısı, Kelime sayısı)

Vektörleştirilmiş veri boyutu: (9632, 5000)


### (3.3) LDA

🕵🏻‍♂️ LDA'yı uyarlayın:
- `n_components = 3` seçin
- *.transform()* ile Konuların Belge Karışımını gösterin
- *.components_* ile Konu Karışımını gösterin

In [11]:
from sklearn.decomposition import LatentDirichletAllocation

# 1. LDA Modelini Kur
# n_components=3 -> "Bana şikayetleri 3 ana başlıkta topla" diyoruz.
lda_model = LatentDirichletAllocation(n_components=3, random_state=42)

# 2. Modeli Eğit ve Dönüştür (Fit & Transform)
# document_mixture: Her bir yorumun konulara olan uzaklığı (Olasılıkları)
document_mixture = lda_model.fit_transform(data_vectorized)

# 3. Konu Bileşenlerini Al (Components)
# topic_mixture: Her bir konunun kelimelerle olan ilişkisi
topic_mixture = lda_model.components_

# 4. Kontrol (Boyutlara bakalım)
print("Belge-Konu Karışımı (Document Mixture):", document_mixture.shape)
print("Konu-Kelime Karışımı (Topic Mixture):", topic_mixture.shape)

Belge-Konu Karışımı (Document Mixture): (9632, 3)
Konu-Kelime Karışımı (Topic Mixture): (3, 5000)


#### Belge Karışımı (Konular için)

In [12]:
import numpy as np

# Her bir yorum için en yüksek puana sahip konuyu (0, 1 veya 2) seçiyoruz
bad_reviews["most_important_topic"] = np.argmax(document_mixture, axis=1)

# Bakalım hangi konudan kaç tane şikayet var?
print(bad_reviews["most_important_topic"].value_counts())

# Örnek olarak ilk 5 satıra bakalım
bad_reviews[["clean_text", "most_important_topic"]].head()

most_important_topic
0    3406
1    3137
2    3089
Name: count, dtype: int64


,clean_text,most_important_topic
14,receb soment control mide split estil falt con...,2
23,compr dua unidad so receb agor fac,1
24,produt bom por vei mim nao condiz fot anunci,2
25,produt inferi mal acab,0
28,entreg praz,1


👉 Her inceleme için en önemli konuyu rapor edelim

In [ ]:
# SENİN KODUN BURAYA

#### Konu Karışımı (Kelimeler için)

In [13]:
# --- Yardımcı Fonksiyonlar ---

def topic_word(vectorizer, model, topic, topwords, with_weights = True):
    # En yüksek ağırlıklı kelimelerin indekslerini bul
    topwords_indexes = topic.argsort()[:-topwords - 1:-1]
    
    # Kelimeleri ve puanlarını getir
    if with_weights == True:
        topwords = [(vectorizer.get_feature_names_out()[i], round(topic[i],2)) for i in topwords_indexes]
    else:
        topwords = [vectorizer.get_feature_names_out()[i] for i in topwords_indexes]
    return topwords

def print_topics(vectorizer, model, topwords):
    # Tüm konuları döngüyle gez ve yazdır
    for idx, topic in enumerate(model.components_):
        print("-" * 20)
        print("Topic %d:" % (idx))
        print(topic_word(vectorizer, model, topic, topwords))

# --- Fonksiyonları Çalıştır ---
# Her konu için en önemli 10 kelimeyi görelim
print_topics(vectorizer, lda_model, 10)

--------------------
Topic 0:
[('nao cheg', np.float64(194.29)), ('produt nao', np.float64(174.57)), ('not fiscal', np.float64(153.11)), ('compr doi', np.float64(120.4)), ('compr dua', np.float64(117.08)), ('receb apen', np.float64(106.38)), ('so entreg', np.float64(104.37)), ('produt compr', np.float64(103.33)), ('receb produt', np.float64(103.02)), ('produt err', np.float64(94.55))]
--------------------
Topic 1:
[('nao receb', np.float64(788.23)), ('receb produt', np.float64(411.62)), ('aind nao', np.float64(340.49)), ('nao entreg', np.float64(320.97)), ('so receb', np.float64(270.17)), ('produt nao', np.float64(253.92)), ('produt entreg', np.float64(222.42)), ('receb apen', np.float64(148.77)), ('dentr praz', np.float64(134.3)), ('ate moment', np.float64(127.02))]
--------------------
Topic 2:
[('produt vei', np.float64(437.78)), ('nao recom', np.float64(230.6)), ('produt cheg', np.float64(183.88)), ('nao gost', np.float64(181.72)), ('receb produt', np.float64(175.36)), ('vei defeit

#### Konular

🎁 Size bazı yardımcı fonksiyonlar sağladık:
- `topic_word`: Tek bir konu (topic) için en önemli kelimeleri ve ağırlıklarını döndürür
- `print_topics`: LDA tarafından bulunan farklı konuları, en önemli kelimeleriyle birlikte yazdırır

In [15]:
def topic_word(vectorizer, model, topic, topwords, with_weights = True):
    topwords_indexes = topic.argsort()[:-topwords - 1:-1]
    if with_weights == True:
        topwords = [(vectorizer.get_feature_names_out()[i], round(topic[i],2)) for i in topwords_indexes]
    if with_weights == False:
        topwords = [vectorizer.get_feature_names_out()[i] for i in topwords_indexes]
    return topwords

In [16]:
def print_topics(vectorizer, model, topwords):
    for idx, topic in enumerate(model.components_):
        print("-"*20)
        print("Topic %d:" % (idx))
        print(topic_word(vectorizer, model, topic, topwords))


🕵🏻‍♂️ Konuları en çok kullanılan kelimelerle birlikte yazdırın:

In [ ]:
# SENİN KODUN BURAYA

🇧🇷 Burada biraz Brezilya Portekizcesi kelimeler var:
- _cadeiras = chairs_
- _produto = product_
- _recomendo = recommend (não recomendo == not recommend)_
- _bom = good_
- _comprei = bought_
- _veio = came_
- _errado = wrong_
- _gostaria = I would like to..._
- _duas = two_
- _nao = not_
- _entregue = delivered_
- _pecas = part_
- _ainda = yet_
- _recebi = received_

👉 Bir konuyla ilişkili en popüler kelimeleri göster

In [18]:
# 1. Her konu için kelime listesini hazırlayalım (Ağırlıklar olmadan, sadece kelimeler)
# Bu liste [ [Konu 0 Kelimeleri], [Konu 1 Kelimeleri], [Konu 2 Kelimeleri] ] şeklinde olacak.
topic_word_mixture = [topic_word(vectorizer, lda_model, topic, 10, with_weights=False) for topic in lda_model.components_]

# 2. Bu kelimeleri ana tablomuzdaki (bad_reviews) her satıra eşleyelim
# Her satırın 'most_important_topic' numarasına bakıp, yukarıdaki listeden kelimeleri çekecek.
bad_reviews["most_important_words"] = bad_reviews["most_important_topic"].apply(lambda i: topic_word_mixture[i])

# 3. Sonucu görelim: Puan, Kategori, Yorum ve Bizim Bulduğumuz Konu Kelimeleri
bad_reviews[[
    "review_score",
    "product_category_name",
    "clean_text",
    "most_important_topic",
    "most_important_words"
]].head()

,review_score,product_category_name,clean_text,most_important_topic,most_important_words
14,1,eletronicos,receb soment control mide split estil falt con...,2,"[produt vei, nao recom, produt cheg, nao gost,..."
23,3,ferramentas_jardim,compr dua unidad so receb agor fac,1,"[nao receb, receb produt, aind nao, nao entreg..."
24,3,beleza_saude,produt bom por vei mim nao condiz fot anunci,2,"[produt vei, nao recom, produt cheg, nao gost,..."
25,1,moveis_decoracao,produt inferi mal acab,0,"[nao cheg, produt nao, not fiscal, compr doi, ..."
28,3,esporte_lazer,entreg praz,1,"[nao receb, receb produt, aind nao, nao entreg..."


In [19]:
df["most_important_words"] = df["most_important_topic"].apply(lambda i: topic_word_mixture[i])

KeyError: 'most_important_topic'

In [ ]:
df[["review_id",
        "review_score",
        "product_category_name",
        "full_review_cleaned",
        "most_important_topic",
        "most_important_words"]
      ].head()

## (3.4) Pipeline Tf-Idf ve LDA

In [20]:
from sklearn import set_config
set_config("diagram")

🔨 Önceki Vectorizer ve LDA'yı birbirine bağlayan bir Pipeline oluşturun.

Temizlenmiş metinlere uyarlayın.

In [21]:
from sklearn.pipeline import make_pipeline

# 1. Pipeline'ı İnşa Et
# İçine sırasıyla işlemleri koyuyoruz: Önce Vectorizer, Sonra LDA
pipeline = make_pipeline(
    CountVectorizer(ngram_range=(2, 2), max_df=0.75, max_features=5000),
    LatentDirichletAllocation(n_components=3, random_state=42)
)

# 2. Pipeline'ı Eğit
# Tek komutla hem vektörleştirir hem de LDA modelini eğitir.
pipeline.fit(bad_reviews['clean_text'])

print("Pipeline başarıyla kuruldu ve eğitildi! 🚀")

Pipeline başarıyla kuruldu ve eğitildi! 🚀


💡 `pipeline.components_` ile bileşenlere erişmeye çalışırsanız, Pipeline'da `components_` olmadığı için bu YÜRÜMEZ. Ancak, LDA'ya erişmek için `pipeline._final_estimator` kullanabilirsiniz. Ve buradan konulara erişebilirsiniz!

In [ ]:
pipeline._final_estimator
# 1. Pipeline içindeki LDA modeline erişim
# _final_estimator bize pipeline'ın son halkasını (LDA modelini) verir.

,"n_components n_components: int, default=10Number of topics... versionchanged:: 0.19 ``n_topics`` was renamed to ``n_components``",3
,"doc_topic_prior doc_topic_prior: float, default=NonePrior of document topic distribution `theta`. If the value is None,defaults to `1 / n_components`.In [1]_, this is called `alpha`.",None
,"topic_word_prior topic_word_prior: float, default=NonePrior of topic word distribution `beta`. If the value is None, defaultsto `1 / n_components`.In [1]_, this is called `eta`.",None
,"learning_method learning_method: {'batch', 'online'}, default='batch'Method used to update `_component`. Only used in :meth:`fit` method.In general, if the data size is large, the online update will be muchfaster than the batch update.Valid options:- 'batch': Batch variational Bayes method. Use all training data in each EM update. Old `components_` will be overwritten in each iteration.- 'online': Online variational Bayes method. In each EM update, use mini-batch of training data to update the ``components_`` variable incrementally. The learning rate is controlled by the ``learning_decay`` and the ``learning_offset`` parameters... versionchanged:: 0.20 The default learning method is now ``""batch""``.",'batch'
,"learning_decay learning_decay: float, default=0.7It is a parameter that control learning rate in the online learningmethod. The value should be set between (0.5, 1.0] to guaranteeasymptotic convergence. When the value is 0.0 and batch_size is``n_samples``, the update method is same as batch learning. In theliterature, this is called kappa.",0.7
,"learning_offset learning_offset: float, default=10.0A (positive) parameter that downweights early iterations in onlinelearning. It should be greater than 1.0. In the literature, this iscalled tau_0.",10.0
,"max_iter max_iter: int, default=10The maximum number of passes over the training data (aka epochs).It only impacts the behavior in the :meth:`fit` method, and not the:meth:`partial_fit` method.",10
,"batch_size batch_size: int, default=128Number of documents to use in each EM iteration. Only used in onlinelearning.",128
,"evaluate_every evaluate_every: int, default=-1How often to evaluate perplexity. Only used in `fit` method.set it to 0 or negative number to not evaluate perplexity intraining at all. Evaluating perplexity can help you check convergencein training process, but it will also increase total training time.Evaluating perplexity in every iteration might increase training timeup to two-fold.",-1
,"total_samples total_samples: int, default=1e6Total number of documents. Only used in the :meth:`partial_fit` method.",1000000.0
,"perp_tol perp_tol: float, default=1e-1Perplexity tolerance. Only used when ``evaluate_every`` is greater than 0.",0.1


In [ ]:
pipeline._final_estimator.components_
# 3. Konu Karışımı (Topic Mixture)
# LDA modelinin içindeki kelime ağırlıkları

array([[9.14959949, 2.32671348, 4.24352861, ..., 4.32774568, 2.75147659,
        4.31070481],
       [0.34426565, 3.33871052, 0.34410201, ..., 0.33376747, 0.36316566,
        0.33791203],
       [0.50613486, 0.334576  , 8.41236938, ..., 0.33848684, 6.88535775,
        0.35138316]], shape=(3, 5000))

Pipeline ile **Belge Karışımı**:

In [24]:
# 2. Belge Karışımı (Document Mixture)
# "Bu metinleri al, pipeline'dan geçir ve konu olasılıklarını ver"
document_mixture_pipe = pipeline.transform(bad_reviews['clean_text'])

Pipeline ile **Konu Karışımı**:

In [26]:
# 1. Pipeline içindeki LDA modeline erişim
# _final_estimator bize pipeline'ın son halkasını (LDA modelini) verir.
lda_model_pipe = pipeline._final_estimator

# 2. Belge Karışımı (Document Mixture)
# "Bu metinleri al, pipeline'dan geçir ve konu olasılıklarını ver"
document_mixture_pipe = pipeline.transform(bad_reviews['clean_text'])

# 3. Konu Karışımı (Topic Mixture)
# LDA modelinin içindeki kelime ağırlıkları
topic_mixture_pipe = lda_model_pipe.components_

# Kontrol
print("Belge Karışımı Boyutu:", document_mixture_pipe.shape)
print("Konu Karışımı Boyutu:", topic_mixture_pipe.shape)

Belge Karışımı Boyutu: (9632, 3)
Konu Karışımı Boyutu: (3, 5000)


## (4) 🎁 Ürün Kategorileri

### (4.1) Ürün kategorilerine göre gruplandırma

📈 Veri kümesini `product_category_name` ile gruplandırın ve performanslarını inceleyin.

In [27]:
# Performansa göre ürün kategorileri - sayı, ortalama, medyan ve standart sapmaya bakın
product_categories = df.groupby(by = 'product_category_name').agg({
        'review_score': ["count", "mean", "median", "std"]
    })

# Analiz için belirli bir süreden daha az satılan ürünleri kaldırma
cutoff = 50
product_categories = product_categories[product_categories[("review_score", "count")] > cutoff]

# Ürün kategorilerini performansa göre sıralama
product_categories = product_categories.sort_values(by = [('review_score', 'mean'),
                                                          ('review_score', 'std')],
                                                    ascending = [False, True])
product_categories

review_score                   \
                                                      count      mean median   
product_category_name                                                          
alimentos_bebidas                                        75  4.560000    5.0   
livros_interesse_geral                                  152  4.539474    5.0   
malas_acessorios                                        375  4.317333    5.0   
livros_tecnicos                                          83  4.301205    5.0   
pcs                                                      67  4.283582    5.0   
alimentos                                               151  4.231788    5.0   
papelaria                                               758  4.201847    5.0   
instrumentos_musicais                                   228  4.192982    5.0   
fashion_calcados                                        101  4.188119    5.0   
fashion_bolsas_e_acessorios                             726  4.168044    5.0   
perfumaria                                             1094  4.167276    5.0   
construcao_ferramentas_jardim                            85  4.164706    5.0   
cool_stuff                                             1360  4.154412    5.0   
ferramentas_jardim                                     1391  4.105679    5.0   
beleza_saude                                           3046  4.104071    5.0   
brinquedos                                             1271  4.101495    5.0   
esporte_lazer                                          2637  4.097838    5.0   
bebidas                                                 111  4.090090    5.0   
pet_shop                                                603  4.086235    5.0   
construcao_ferramentas_iluminacao                        86  4.081395    5.0   
eletroportateis                                         242  4.061983    5.0   
automotivo                                             1533  4.049576    5.0   
eletrodomesticos                                        354  4.033898    5.0   
eletrodomesticos_2                                       89  4.011236    5.0   
relogios_presentes                                     2211  4.009950    5.0   
construcao_ferramentas_construcao                       277  4.007220    5.0   
consoles_games                                          339  4.005900    5.0   
utilidades_domesticas                                  2192  4.005018    5.0   
eletronicos                                             942  3.979830    5.0   
industria_comercio_e_negocios                            82  3.963415    5.0   
bebes                                                   882  3.955782    5.0   
sinalizacao_e_seguranca                                  61  3.934426    5.0   
telefonia                                              1654  3.874244    5.0   
cama_mesa_banho                                        3891  3.853765    5.0   
artes                                                    80  3.850000    5.0   
casa_conforto                                           175  3.817143    5.0   
moveis_decoracao                                       2311  3.815232    5.0   
moveis_sala                                             138  3.789855    5.0   
informatica_acessorios                                 2473  3.774767    5.0   
audio                                                   134  3.768657    5.0   
market_place                                            102  3.745098    4.0   
moveis_cozinha_area_de_servico_jantar_e_jardim           94  3.734043    4.0   
construcao_ferramentas_seguranca                         61  3.721311    5.0   
agro_industria_e_comercio                                57  3.701754    4.0   
telefonia_fixa                                           96  3.697917    4.0   
casa_construcao                                         177  3.672316    4.0   
climatizacao                                             93  3.655914    4.0   
fashion_roupa_masculina                        

### (4.2) En kötü ürün kategorileri

👎 *Ortalama değerlendirme puanı* açısından en kötü beş kategoriyi `worst_products` adlı bir değişkene kaydedin.

In [28]:
worst_products = product_categories.tail(5).sort_values(by = [("review_score", "count")],
                                                       ascending = False)
worst_products

review_score                           
                               count      mean median       std
product_category_name                                          
moveis_escritorio                546  3.298535    4.0  1.562366
casa_construcao                  177  3.672316    4.0  1.498094
telefonia_fixa                    96  3.697917    4.0  1.576938
climatizacao                      93  3.655914    4.0  1.625173
fashion_roupa_masculina           56  3.482143    4.0  1.737198

👇 Yalnızca `worst_products` öğelerini içeren bir `worst_products_review` DataFrame oluşturun.

In [29]:
worst_products_reviews = df[df.product_category_name.isin(worst_products.index)]
worst_products_reviews[["review_id",
                        "review_score",
                        "product_category_name",
                        "full_review_cleaned",
                        "most_important_topic",
                        "most_important_words"]
      ]

KeyError: "['full_review_cleaned', 'most_important_topic', 'most_important_words'] not in index"

In [30]:
# --- 1. Kategorileri Performansa Göre Sırala ---
# Her kategori için yorum sayısı, ortalama puan ve standart sapmayı hesapla
product_categories = df.groupby(by='product_category_name').agg({
    'review_score': ["count", "mean", "median", "std"]
})

# Güvenilir sonuç için en az 50 yorumu olan kategorileri alalım (Az yorum yanıltıcı olabilir)
cutoff = 50
product_categories = product_categories[product_categories[("review_score", "count")] > cutoff]

# En kötüden en iyiye doğru değil, genel bir sıralama yapalım
# (Not defterinde sıralama mantığı biraz farklı olabilir ama biz ortalamaya bakalım)
product_categories = product_categories.sort_values(by=[('review_score', 'mean')], ascending=True)

# --- 2. En Kötü 5 Kategoriyi Seç (Worst Products) ---
# Ortalaması en düşük olan ilk 5 kategori
worst_products = product_categories.head(5)
print("En Kötü 5 Kategori:")
print(worst_products)

# --- 3. Bu Kategorilerin Yorumlarını Getir (Worst Products Reviews) ---
# Sadece bu 5 kategoriye ait yorumları filtrele
worst_products_reviews = df[df.product_category_name.isin(worst_products.index)]

# Kontrol edelim
print(f"\nİncelenecek şikayet sayısı: {len(worst_products_reviews)}")
worst_products_reviews.head()

En Kötü 5 Kategori:
                        review_score                           
                               count      mean median       std
product_category_name                                          
moveis_escritorio                546  3.298535    4.0  1.562366
fashion_roupa_masculina           56  3.482143    4.0  1.737198
climatizacao                      93  3.655914    4.0  1.625173
casa_construcao                  177  3.672316    4.0  1.498094
telefonia_fixa                    96  3.697917    4.0  1.576938

İncelenecek şikayet sayısı: 968


,review_id,length_review,review_score,order_id,product_category_name,full_review,clean_text
45,c422274e50e900b46fee429016c5458d,7,3,950bb159eea525ff8947b92c93b02f8e,moveis_escritorio,Gostei,gost
126,5d6f9cddc8335878d8bf20c3bd4602e8,162,5,38fc895ea0a2aa253a46a6fdbb65aaf5,moveis_escritorio,mega recomendo Recebi meu produto corretamente...,meg recom receb produt corret dentr prazoverif...
212,1f836d160f50247a394034abd93b7c24,22,3,7951fe56cf685dca47524b1986bf56b6,casa_construcao,Esperando o instalador,esper instal
219,edd86e0af4dc62b340c60d846643cbf9,127,5,9537e35e835d4ae6b7b219b8b2e36817,moveis_escritorio,"PRODUTO OK prazo foi estendido,porém entregue ...",produt ok praz estendidopor entreg praz divulg...
232,e47020cf52099bce9dc57524ea41218b,42,4,2e23a32f0160b8226732e1fab87cbe59,moveis_escritorio,Indico a loja compro sempre na lannister .,indic loj compr sempr lannist


### (4.3). En kötü ürünler için konular

❓ En kötü ürünlerin konuları nelerdir? ❓

In [31]:
worst_products_reviews["most_important_topic"].value_counts()

KeyError: 'most_important_topic'

In [ ]:
bad_frequency = list(worst_products_reviews["most_important_topic"].value_counts().index)
bad_frequency

In [ ]:
[topic_word_mixture[i] for i in bad_frequency]

In [32]:
# 1. En kötü kategorilerin isimlerini alalım
worst_category_names = worst_products.index

# 2. bad_reviews tablosunu sadece bu kategoriler için filtreleyelim
# (Çünkü konu etiketleri bad_reviews tablosunda hazır duruyor)
worst_category_bad_reviews = bad_reviews[bad_reviews['product_category_name'].isin(worst_category_names)]

# 3. Bu kategorilerde en çok hangi konu konuşulmuş? Sayalım.
topic_counts = worst_category_bad_reviews['most_important_topic'].value_counts()
print("Konu Dağılımı:")
print(topic_counts)

# 4. En sık görülen konunun kelimelerini görelim
top_topic_id = topic_counts.idxmax()
print(f"\nEn sık görülen şikayet konusu: Topic {top_topic_id}")
print("Bu konunun kelimeleri:", topic_word_mixture[top_topic_id])

Konu Dağılımı:
most_important_topic
0    147
2    144
1    122
Name: count, dtype: int64

En sık görülen şikayet konusu: Topic 0
Bu konunun kelimeleri: ['nao cheg', 'produt nao', 'not fiscal', 'compr doi', 'compr dua', 'receb apen', 'so entreg', 'produt compr', 'receb produt', 'produt err']


## (5) 🎁 Satıcılar...

* En kötü satıcılar tarafından ne tür ürünler satıldı?
* En kötü satıcılar için başlıca yorumlar nelerdir?

### (5.1) En kötü satıcılar

In [33]:
from olist.seller import Seller
sellers = Seller().get_training_data()
sellers.columns

/home/tumay/code/data-context-and-setup/olist/seller.py:64: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(delay_to_logistic_partner)\
/home/tumay/code/data-context-and-setup/olist/seller.py:69: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(order_wait_time)\


Index(['seller_id', 'seller_city', 'seller_state', 'delay_to_carrier',
       'wait_time', 'date_first_sale', 'date_last_sale', 'months_on_olist',
       'n_orders', 'quantity', 'quantity_per_order', 'sales',
       'share_of_one_stars', 'share_of_five_stars', 'review_score'],
      dtype='object')

In [36]:
from olist.seller import Seller
sellers = Seller().get_training_data()
print(sellers.columns)

/home/tumay/code/data-context-and-setup/olist/seller.py:64: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(delay_to_logistic_partner)\
/home/tumay/code/data-context-and-setup/olist/seller.py:69: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(order_wait_time)\


Index(['seller_id', 'seller_city', 'seller_state', 'delay_to_carrier',
       'wait_time', 'date_first_sale', 'date_last_sale', 'months_on_olist',
       'n_orders', 'quantity', 'quantity_per_order', 'sales',
       'share_of_one_stars', 'share_of_five_stars', 'review_score'],
      dtype='object')


👇 En kötü satan 10 ürünü seçin ve bunları `worst_sellers` adlı bir değişkene kaydedin.

In [37]:
# 1. Satıcı verilerini çek (Uyarıları şimdilik görmezden gel)
sellers = Seller().get_training_data()

# 2. "review_score"a göre sırala (Küçükten büyüğe)
# Puanı en düşük olan (1 yıldız) satıcılar en başa gelir.
worst_sellers = sellers[["seller_id", "review_score", "sales"]].sort_values(
    by="review_score",
    ascending=True
).head(10)

# Sonucu görelim
worst_sellers

/home/tumay/code/data-context-and-setup/olist/seller.py:64: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(delay_to_logistic_partner)\
/home/tumay/code/data-context-and-setup/olist/seller.py:69: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(order_wait_time)\


,seller_id,review_score,sales
15,1444c08e64d55fb3c25f0f09c07ffcf2,1.0,2749.00
2951,4e2627090e6e5b9fabba883a37897683,1.0,199.50
2950,63704069d9bd3a75c1cf59babe56004a,1.0,59.30
4,51a04a8a6bdcb23deccc82b0b80742cf,1.0,167.99
957,c004e5ea15737026cecaee0447e00b75,1.0,4700.00
944,553a440137d6e0cf776aee0bb83ac9f5,1.0,144.80
984,9c57bc60cfad5ee62d35d3f1ce4593a1,1.0,180.00
958,8629a7efec1aab257e58cda559f03ba7,1.0,45.22
153,c1dde11f12d05c478f5de2d7319ad3b2,1.0,12.50
1487,96f7c797de9ca20efbe14545bed63eec,1.0,138.60


### (5.2) En kötü satıcılar tarafından satılan ürünler

In [45]:
products = Product().get_training_data() [["product_id", "category"]]
products

,product_id,category
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,art
2,96bd76ec8810374ed1b65e291975717f,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,baby
4,9dc1a7de274444849c219cff195d0b71,housewares
...,...,...
31479,a0b7d5a992ccda646f2d34e418fff5a0,furniture_decor
31480,bf4538d88321d0fd4412a93c974510e6,construction_tools_lights
31481,9a7c6041fa9592d9d9ef6cfe62a71f8c,bed_bath_table
31482,83808703fc0706a22e264b9d75f04a2e,computers_accessories


In [49]:
import sys
sys.path.insert(0, "/home/tumay/code/data-context-and-setup")

In [52]:
from olist.data import Olist
# Tüm verileri (order_items, products, sellers vb.) data değişkenine yüklüyoruz
data = Olist().get_data()

print("✅ Data kutusu hazır!")

✅ Data kutusu hazır!


❓ En kötü satıcılar tarafından satılan ürün türleri nelerdir? ❓

In [53]:
sellers_product_category = data["order_items"].merge(products,
                                             on = "product_id", how = "left")[["seller_id", "category"]]

sellers_product_category

,seller_id,category
0,48436dade18ac8b2bce089ec2a041202,cool_stuff
1,dd7ddc04e1b6c2c614352b383efe2d36,pet_shop
2,5b51032eddd242adc84c38acab88f23d,furniture_decor
3,9d7a1d34a5052409006425275ba1c2b4,perfumery
4,df560393f3a51e74553ab94004ba5c87,garden_tools
...,...,...
112645,b8bc237ba3788b23da09c0f1f3a3288c,housewares
112646,f3c38ab652836d21de61fb8314b69182,computers_accessories
112647,c3cfdc648177fdbbbb35635a37472c53,sports_leisure
112648,2b3e4a2a3ea8e01938cabda2a3e5cc79,computers_accessories


In [54]:
sellers_product_category.groupby("seller_id").count()

,category
seller_id,
0015a82c2db000af6aaaf3ae2ecb0532,3
001cca7ae9ae17fb1caed9dfb1094831,239
001e6ad469a905060d959994f1b41e4f,0
002100f778ceb8431b7a1020ff7ab48f,55
003554e2dce176b5555353e4f3555ac8,0
...,...
ffcfefa19b08742c5d315f2791395ee5,0
ffdd9f82b9a447f6f8d4b91554cc7dd3,20
ffeee66ac5d5a62fe688b9d26f83f534,14


### (5.3) En kötü satıcılar için kategoriler ve konular

🎁 İşte bazı kullanışlı işlevler:
- Bir satıcı tarafından satılan ürün kategorilerini göstermek için `focus_seller(seller_id)`
- Bir satıcı için en sık kullanılan konuların en popüler kelimelerini göstermek için `bad_reviews_seller`

In [55]:
def focus_seller(seller_id):
    return sellers_product_category[sellers_product_category.seller_id == seller_id].value_counts()

In [56]:
bad_reviews_sellers = worst_products_reviews.merge(data["order_items"])
bad_reviews_sellers.head(3)

,review_id,length_review,review_score,order_id,product_category_name,full_review,clean_text,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,c422274e50e900b46fee429016c5458d,7,3,950bb159eea525ff8947b92c93b02f8e,moveis_escritorio,Gostei,gost,1,86f2416d4670e4ea3ca5494d043d9f24,7c67e1448b00f6e969d365cea6b010ab,2017-09-22 04:55:08,159.94,25.63
1,5d6f9cddc8335878d8bf20c3bd4602e8,162,5,38fc895ea0a2aa253a46a6fdbb65aaf5,moveis_escritorio,mega recomendo Recebi meu produto corretamente...,meg recom receb produt corret dentr prazoverif...,1,2a5806f10d0f00e5ad032dd2e3c8806e,7c67e1448b00f6e969d365cea6b010ab,2018-08-09 10:24:11,169.99,34.27
2,1f836d160f50247a394034abd93b7c24,22,3,7951fe56cf685dca47524b1986bf56b6,casa_construcao,Esperando o instalador,esper instal,1,5926c58c462b5d1f932ef3f0a9e37d41,c64a2aec32cc408a8a4c6d7c46017f91,2017-12-27 23:14:26,386.75,63.17


In [57]:
def bad_reviews_seller(bad_reviews_sellers, seller_id):
    mask = (bad_reviews_sellers.seller_id == seller_id)
    temp = bad_reviews_sellers[mask]
    if len(temp) > 0: # satıcı kötü yorumlar veri çerçevesinde görünüyorsa
        most_frequent_topic_seller = list(temp.most_important_topic.value_counts().head(1).index)[0]
        return topic_word_mixture[most_frequent_topic_seller]

❓Bu en az satan ürünlerin her biri için en sık kullanılan ürün kategorilerini ve kelimeleri gösterin ❓

In [58]:
for worst_seller in worst_sellers["seller_id"]:
    print("-"*50)
    print(f"Focusing on the seller #{worst_seller}...")
    print(focus_seller(worst_seller))
    print(bad_reviews_seller(bad_reviews_sellers, worst_seller))


--------------------------------------------------
Focusing on the seller #1444c08e64d55fb3c25f0f09c07ffcf2...
seller_id                         category    
1444c08e64d55fb3c25f0f09c07ffcf2  garden_tools    1
Name: count, dtype: int64
None
--------------------------------------------------
Focusing on the seller #4e2627090e6e5b9fabba883a37897683...
Series([], Name: count, dtype: int64)
None
--------------------------------------------------
Focusing on the seller #63704069d9bd3a75c1cf59babe56004a...
seller_id                         category
63704069d9bd3a75c1cf59babe56004a  drinks      1
Name: count, dtype: int64
None
--------------------------------------------------
Focusing on the seller #51a04a8a6bdcb23deccc82b0b80742cf...
seller_id                         category   
51a04a8a6bdcb23deccc82b0b80742cf  electronics    1
Name: count, dtype: int64
None
--------------------------------------------------
Focusing on the seller #c004e5ea15737026cecaee0447e00b75...
seller_id             

🏁 Tebrikler. NLP'nin bazı temellerini (Ön İşleme + Vektörleştirme + NB/LDA) öğrendiniz ve bu yeni “uzmanlığı” Karar Bilimi ile birleştirdik.

💾 `git add / commit / push` yapmayı unutmayın.